### Commands

In [1]:
from firecloud import api as firecloud_api
import fc_interface

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [3]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

<Response [200]>

In [17]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

<Response [200]>

In [42]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [16]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [18]:
### Upload pairs
# res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [48]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [70]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

<Response [204]>

In [74]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [94]:
### Create cumulative PoN
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'CumPoN21')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

Creating panel of 157 normals


In [109]:
### Create PoN from normals of only one batch
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

Creating panel of 11 normals


<Response [200]>

In [102]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

<Response [204]>

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [20]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [145]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

---

---

In [4]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)

In [38]:
all_paths_df = pd.read_excel('paths_to_batches_info.xlsx')

In [47]:
for idx, row in all_paths_df.iterrows():
    fc_interface.prepare_batch_sample_set_for_metadata_export(row.path_to_samples_info, row.tsca_id);
    tsca_id = row.tsca_id
    tumors_sample_set_metadata  = "%s/fc_upload_sample_set_tsca_%s_tumors.txt" % (tsca_id, tsca_id)
    normals_sample_set_metadata = "%s/fc_upload_sample_set_tsca_%s_normals.txt" % (tsca_id, tsca_id)
#     res = fc_interface.upload_entities_from_tsv(namespace, workspace, tumors_sample_set_metadata)
#     res = fc_interface.upload_entities_from_tsv(namespace, workspace, normals_sample_set_metadata)

190 Samples in this batch
95 Samples in this batch
190 Samples in this batch
95 Samples in this batch
95 Samples in this batch
95 Samples in this batch
95 Samples in this batch
95 Samples in this batch
95 Samples in this batch


In [46]:
fc_interface.prepare_batch_sample_set_for_metadata_export(all_paths[-1], 'TSCA21');

95 Samples in this batch


ValueError: labels ['sample_type'] not contained in axis